In [ ]:
import numpy as np
import glob
import math
import sys 
import os
from math import pi
import matplotlib.pyplot as plt
from scipy.linalg import expm
from scipy.linalg import eigh
import sympy as sp
%matplotlib inline
from matplotlib.ticker import ScalarFormatter
import h5py


In [ ]:
#pip install jax jaxlib

In [ ]:
import jax
import jax.numpy as np
from jax import jit, grad, vmap
from jax.scipy.optimize import minimize
from functools import partial

In [ ]:

from py_alf import ALF_source, Simulation # Interface with ALF
#alf_src = ALF_source(branch='master') 
alf_src = ALF_source(branch='VAFQMC') 

In [ ]:


def setup_simulations(alf_src, bin_values):
    for NBin in bin_values:
        sim_params = {
            'Model': 'Hubbard_optimization',
            'Lattice_type': 'Square',
            'L1': 4,
            'L2': 4,
            'Symm': True,
            'ham_T': 1.0,
            'ham_U': 2.0,
            'ham_chem': 0.0,
            'beta': 2.0,
            'Ltau': 0,
            'NSweep': 10,
            'NBin': NBin,  # Use the current bin value
            'dtau': 0.1,
            'Projector': True,
            'Theta': 10.0,
            'Adiabatic': True,
        }

        sim = Simulation(alf_src, 'Hubbard_optimization', sim_params)
        sim.compile()
        sim.run()

# Define bin values
bin_values = [1, 2, 3, 4, 5]

# Create an instance of ALF_source (assuming it's a class)
alf_src_instance = ALF_source()

# Set up simulations
setup_simulations(alf_src_instance, bin_values)



# Define the file path
file_path = "ALF_data/Hubbard_optimization_*/data.h5"

# Initialize lists to store data
obser_data_list = []
sign_data_list = []

# Use os.path to find matching files
matching_files = glob.glob(file_path)

# Ensure that at least one file matches the pattern
if matching_files:
    # Loop over each matching file
    for file in matching_files:
        # Open the HDF5 file
        with h5py.File(file, 'r') as f:
            # Access the Ener_scal group
            Ener_scal_group = f['Ener_scal']
            
            # Get the data from the 'obser' and 'sign' datasets
            obser_data = Ener_scal_group['obser'][:]
            sign_data = Ener_scal_group['sign'][:]
            
            # Store the data in lists
            obser_data_list.append(obser_data)
            sign_data_list.append(sign_data)
else:
    print("No matching HDF5 files found.")


# Print values in obser_data_list
print("Values in obser_data_list:")
for obser_data in obser_data_list:
    print(obser_data)

# Print values in sign_data_list
print("\nValues in sign_data_list:")
for sign_data in sign_data_list:
    print(sign_data)

# Plotting
plt.figure(figsize=(10, 5))

# Plot 'obser' data

# Flatten obser_data_list to ensure compatibility with index
flat_obser_data = np.concatenate(obser_data_list)

# Flatten sign_data_list to ensure compatibility with index
flat_sign_data = np.concatenate(sign_data_list)

# Plot index versus 'obser' data
plt.figure(figsize=(10, 5))
plt.plot(range(len(flat_obser_data)), flat_obser_data[:, 0, 0], label='obser')

# Plot index versus 'sign' data
#plt.plot(range(len(flat_sign_data)), flat_sign_data, label='sign', color='orange')

plt.xlabel('Index')
plt.ylabel('Value')
plt.title('Index vs Data')
plt.legend()
plt.show()
